# Low-rank approximation on $\mathcal{P}(d)$ - the space of $d$-dimensional SPD matrices

In this notebook we want to get some intuition in different approaches for computing low-rank approximations for manifold-valued signals

In [29]:
using Manifolds
using Manopt
using LinearAlgebra
using Random
using Plots
using LaTeXStrings
using BenchmarkTools

In [3]:
include("../../../src/decompositions/signals/naive_low_rank_approximation.jl")
include("../../../src/decompositions/signals/curvature_corrected_low_rank_approximation.jl")
include("../../../src/decompositions/signals/exact_low_rank_approximation.jl")
include("../../../src/decompositions/signals/stochastic_curvature_corrected_low_rank_approximation.jl")

include("../../../src/functions/loss_functions/curvature_corrected_loss.jl")
include("../../../src/functions/loss_functions/exact_loss.jl")
include("../../../src/functions/jacobi_field/beta.jl")

β (generic function with 1 method)

In [4]:
black = RGBA{Float64}(colorant"#000000")
TolVibrantOrange = RGBA{Float64}(colorant"#EE7733")
TolVibrantBlue = RGBA{Float64}(colorant"#0077BB")
TolVibrantTeal = RGBA{Float64}(colorant"#009988")
TolVibrantMagenta = RGBA{Float64}(colorant"#EE3377")
TolVibrantCyan = RGBA{Float64}(colorant"#33BBEE");
render_size = 1

1

### Load data and construct manifold ###

In [5]:
# load data
M = Sphere(6)
d = manifold_dimension(M)
n = 100  # 100


100

In [6]:
θ = collect(range(0, 2 * π, n+1))[1:n]
predata = [[cos(θ[i]), sin(θ[i]), 0., 0., 0., 0., 0.] for i in 1:n]
σ = .05  # variance

Random.seed!(31)
data = [exp(M, predata[i], random_tangent(M, predata[i], Val(:Gaussian), σ)) for i in 1:n]; # ∈ S(6)^n

In [34]:
# Export slice image
# asymptote_export_S2_signals("results/artificial1D_orig.asy", points=[data],
# colors=Dict(:points => [TolVibrantBlue]),
# dot_size=3.5,
# camera_position=(1.0, 0.5, 0.5)); 

### Construct low rank approximation ###

In [7]:
q₁ = mean(M, data)
println(q₁)
log_q_data₁ = log.(Ref(M), Ref(q₁), data);  # ∈ T_q P(3)^n
# TODO compute theoretical lower bound
ONB₁ = get_basis.(Ref(M), Ref(q₁), DiagonalizingOrthonormalBasis.(log_q_data₁))
κ₁ = maximum([maximum(distance(M, q₁, data[i])^2 .* ONB₁[i].data.eigenvalues) for i in 1:n])
println(κ₁)


summed_pairwise_distances = [sum([distance.(Ref(M), Ref(data[i]), data) .^2]) for i in 1:n]
q₂ = data[argmin(summed_pairwise_distances)]
log_q_data₂ = log.(Ref(M), Ref(q₂), data);  # ∈ T_q P(3)^n
println(q₂)
ONB₂ = get_basis.(Ref(M), Ref(q₂), DiagonalizingOrthonormalBasis.(log_q_data₂))
κ₂ = maximum([maximum(distance(M, q₂, data[i])^2 .* ONB₂[i].data.eigenvalues) for i in 1:n])
println(κ₂)

[0.011362771023773281, -0.014665511087507247, 0.32104150872364784, -0.4013489917107375, -0.07156976421128587, -0.852341865783546, -0.06243604273002383]
2.870971878384671
[0.9944358300450549, -0.05035588364543773, 0.05181675794418579, 0.03470388913095954, 0.013728091849230824, 0.009286850030275141, -0.06631457223660438]
9.16012544722712


In [12]:
exact_low_rank_approximation(M, q₁, data, 1; stepsize=1/64, max_iter=50, print_iterates=true);

Initial  F(x): 0.40744920354 | 
# 1     change: 0.070133459 |  F(x): 0.40640345319 | 
# 2     change: 0.068347618 |  F(x): 0.40541838533 | 
# 3     change: 0.066607254 |  F(x): 0.40449074930 | 
# 4     change: 0.064911216 |  F(x): 0.40361746224 | 
# 5     change: 0.063258380 |  F(x): 0.40279560058 | 
# 6     change: 0.061647653 |  F(x): 0.40202239204 | 
# 7     change: 0.060077964 |  F(x): 0.40129520794 | 
# 8     change: 0.058548274 |  F(x): 0.40061155597 | 
# 9     change: 0.057057566 |  F(x): 0.39996907327 | 
# 10    change: 0.055604850 |  F(x): 0.39936551985 | 
# 11    change: 0.054189160 |  F(x): 0.39879877237 | 
# 12    change: 0.052809555 |  F(x): 0.39826681818 | 
# 13    change: 0.051465116 |  F(x): 0.39776774973 | 
# 14    change: 0.050154949 |  F(x): 0.39729975919 | 
# 15    change: 0.048878180 |  F(x): 0.39686113334 | 
# 16    change: 0.047633961 |  F(x): 0.39645024880 | 
# 17    change: 0.046421461 |  F(x): 0.39606556739 | 
# 18    change: 0.045239872 |  F(x): 0.39570563180

In [21]:
exact_low_rank_approximation(M, q₂, data, 1; stepsize=1/16, max_iter=50, print_iterates=true); 

Initial  F(x): 0.04339286801 | 
# 1     change: 0.121951793 |  F(x): 0.04249062289 | 
# 2     change: 0.108072799 |  F(x): 0.04176749479 | 
# 3     change: 0.095791239 |  F(x): 0.04118662614 | 
# 4     change: 0.084924037 |  F(x): 0.04071891344 | 
# 5     change: 0.075309265 |  F(x): 0.04034135480 | 
# 6     change: 0.066803682 |  F(x): 0.04003575241 | 
# 7     change: 0.059280571 |  F(x): 0.03978769314 | 
# 8     change: 0.052627821 |  F(x): 0.03958574756 | 
# 9     change: 0.046746238 |  F(x): 0.03942084015 | 
# 10    change: 0.041548050 |  F(x): 0.03928575390 | 
# 11    change: 0.036955585 |  F(x): 0.03917474058 | 
# 12    change: 0.032900095 |  F(x): 0.03908321376 | 
# 13    change: 0.029320726 |  F(x): 0.03900750707 | 
# 14    change: 0.026163593 |  F(x): 0.03894468354 | 
# 15    change: 0.023380962 |  F(x): 0.03889238526 | 
# 16    change: 0.020930526 |  F(x): 0.03884871470 | 
# 17    change: 0.018774757 |  F(x): 0.03881214081 | 
# 18    change: 0.016880331 |  F(x): 0.03878142495

In [22]:
max_iter = 50

nR_q₁ = []
nU₁ = []
ccR_q₁ = []
ccU₁ = []
eR_q₁ = []
eU₁ = []
eCosts₁ = []

nR_q₂ = []
nU₂ = []
ccR_q₂ = []
ccU₂ = []
eR_q₂ = []
eU₂ = []
eCosts₂ = []

for i in 1:d
    println("#$(i) | computing naive low-rank approximation")
    nRr_q₁, nUr₁ = naive_low_rank_approximation(M, q₁, data, i)
    push!(nR_q₁, nRr_q₁)
    push!(nU₁, nUr₁)
    println("#$(i) | computing curvature corrected low-rank approximation")
    ccRr_q₁, ccUr₁ = curvature_corrected_low_rank_approximation(M, q₁, data, i); 
    push!(ccR_q₁, ccRr_q₁)
    push!(ccU₁, ccUr₁)
    println("#$(i) | computing exact low-rank approximation")
    (eRr_q₁, eUr₁), eCostsr₁ = exact_low_rank_approximation(M, q₁, data, i; stepsize=1/64, max_iter=max_iter); 
    push!(eR_q₁, eRr_q₁)
    push!(eU₁, eUr₁)
    push!(eCosts₁, eCostsr₁)

    println("#$(i) | computing naive low-rank approximation")
    nRr_q₂, nUr₂ = naive_low_rank_approximation(M, q₂, data, i)
    push!(nR_q₂, nRr_q₂)
    push!(nU₂, nUr₂)
    println("#$(i) | computing curvature corrected low-rank approximation")
    ccRr_q₂, ccUr₂ = curvature_corrected_low_rank_approximation(M, q₂, data, i); 
    push!(ccR_q₂, ccRr_q₂)
    push!(ccU₂, ccUr₂)
    println("#$(i) | computing exact low-rank approximation")
    (eRr_q₂, eUr₂), eCostsr₂ = exact_low_rank_approximation(M, q₂, data, i; stepsize=1/16, max_iter=max_iter); 
    push!(eR_q₂, eRr_q₂)
    push!(eU₂, eUr₂)
    push!(eCosts₂, eCostsr₂)
end

#1 | computing naive low-rank approximation
#1 | computing curvature corrected low-rank approximation
#1 | computing exact low-rank approximation
Initial  F(x): 0.40744920354 | 
# 1     change: 0.070133459 |  F(x): 0.40640345319 | 
# 2     change: 0.068347618 |  F(x): 0.40541838533 | 
# 3     change: 0.066607254 |  F(x): 0.40449074930 | 
# 4     change: 0.064911216 |  F(x): 0.40361746224 | 
# 5     change: 0.063258380 |  F(x): 0.40279560058 | 
# 6     change: 0.061647653 |  F(x): 0.40202239204 | 
# 7     change: 0.060077964 |  F(x): 0.40129520794 | 
# 8     change: 0.058548274 |  F(x): 0.40061155597 | 
# 9     change: 0.057057566 |  F(x): 0.39996907327 | 
# 10    change: 0.055604850 |  F(x): 0.39936551985 | 
# 11    change: 0.054189160 |  F(x): 0.39879877237 | 
# 12    change: 0.052809555 |  F(x): 0.39826681818 | 
# 13    change: 0.051465116 |  F(x): 0.39776774973 | 
# 14    change: 0.050154949 |  F(x): 0.39729975919 | 
# 15    change: 0.048878180 |  F(x): 0.39686113334 | 
# 16    chan

In [23]:
# basepoint q₁
ref_distance₁ = sum(distance.(Ref(M), Ref(q₁), data).^2)

naive_tangent_distances_r₁ = zeros(d)
predicted_naive_distances_r₁ = zeros(d)
true_naive_distances_r₁ = zeros(d)

curvature_corrected_tangent_distances_r₁ = zeros(d)
predicted_curvature_corrected_distances_r₁ = zeros(d)
true_curvature_corrected_distances_r₁ = zeros(d)

exact_tangent_distances_r₁ = zeros(d)
exact_distances_r₁ = zeros(d)

# basepoint q₂
ref_distance₂ = sum(distance.(Ref(M), Ref(q₂), data).^2)

naive_tangent_distances_r₂ = zeros(d)
predicted_naive_distances_r₂ = zeros(d)
true_naive_distances_r₂ = zeros(d)

curvature_corrected_tangent_distances_r₂ = zeros(d)
predicted_curvature_corrected_distances_r₂ = zeros(d)
true_curvature_corrected_distances_r₂ = zeros(d)

exact_tangent_distances_r₂ = zeros(d)
exact_distances_r₂ = zeros(d)

for rank in 1:d
    ## basepoint q₁
    naive_log_q_data_r₁ = [sum([nR_q₁[rank][i] * nU₁[rank][k,i] for i in 1:rank]) for k in 1:n]
    curvature_corrected_log_q_data_r₁ = [sum([ccR_q₁[rank][i] * ccU₁[rank][k,i] for i in 1:rank]) for k in 1:n]
    exact_log_q_data_r₁ = [sum([eR_q₁[rank][i] * eU₁[rank][k,i] for i in 1:rank]) for k in 1:n]
    
    # expoentiate back
    naive_data_r₁ = exp.(Ref(M), Ref(q₁), naive_log_q_data_r₁)
    curvature_corrected_data_r₁ = exp.(Ref(M), Ref(q₁), curvature_corrected_log_q_data_r₁)
    exact_data_r₁ = exp.(Ref(M), Ref(q₁), exact_log_q_data_r₁)


    # compute relative tangent space error
    naive_tangent_distances_r₁[rank] = sum(norm.(Ref(M), Ref(q₁),  log_q_data₁ - naive_log_q_data_r₁).^2) / ref_distance₁
    curvature_corrected_tangent_distances_r₁[rank] = sum(norm.(Ref(M), Ref(q₁),  log_q_data₁ - curvature_corrected_log_q_data_r₁).^2) / ref_distance₁
    exact_tangent_distances_r₁[rank] = sum(norm.(Ref(M), Ref(q₁),  log_q_data₁ - exact_log_q_data_r₁).^2) / ref_distance₁


    # compute relative manifold error
    predicted_naive_distances_r₁[rank] = curvature_corrected_loss(M, q₁, data, naive_log_q_data_r₁)
    true_naive_distances_r₁[rank] = exact_loss(M, q₁, data, naive_log_q_data_r₁)
    predicted_curvature_corrected_distances_r₁[rank] = curvature_corrected_loss(M, q₁, data, curvature_corrected_log_q_data_r₁)
    true_curvature_corrected_distances_r₁[rank] = exact_loss(M, q₁, data, curvature_corrected_log_q_data_r₁)
    exact_distances_r₁[rank] = exact_loss(M, q₁, data, exact_log_q_data_r₁)

    ## basepoint q₂
    naive_log_q_data_r₂ = [sum([nR_q₂[rank][i] * nU₂[rank][k,i] for i in 1:rank]) for k in 1:n]
    curvature_corrected_log_q_data_r₂ = [sum([ccR_q₂[rank][i] * ccU₂[rank][k,i] for i in 1:rank]) for k in 1:n]
    exact_log_q_data_r₂ = [sum([eR_q₂[rank][i] * eU₂[rank][k,i] for i in 1:rank]) for k in 1:n]
    
    # exponentiate back
    naive_data_r₂ = exp.(Ref(M), Ref(q₂), naive_log_q_data_r₂)
    curvature_corrected_data_r₂ = exp.(Ref(M), Ref(q₂), curvature_corrected_log_q_data_r₂)
    exact_data_r₂ = exp.(Ref(M), Ref(q₂), exact_log_q_data_r₂)


    # compute relative tangent space error
    naive_tangent_distances_r₂[rank] = sum(norm.(Ref(M), Ref(q₂),  log_q_data₂ - naive_log_q_data_r₂).^2) / ref_distance₂
    curvature_corrected_tangent_distances_r₂[rank] = sum(norm.(Ref(M), Ref(q₂),  log_q_data₂ - curvature_corrected_log_q_data_r₂).^2) / ref_distance₂
    exact_tangent_distances_r₂[rank] = sum(norm.(Ref(M), Ref(q₂),  log_q_data₂ - exact_log_q_data_r₂).^2) / ref_distance₂


    # compute relative manifold error
    predicted_naive_distances_r₂[rank] = curvature_corrected_loss(M, q₂, data, naive_log_q_data_r₂)
    true_naive_distances_r₂[rank] = exact_loss(M, q₂, data, naive_log_q_data_r₂)
    predicted_curvature_corrected_distances_r₂[rank] = curvature_corrected_loss(M, q₂, data, curvature_corrected_log_q_data_r₂)
    true_curvature_corrected_distances_r₂[rank] = exact_loss(M, q₂, data, curvature_corrected_log_q_data_r₂)
    exact_distances_r₂[rank] = exact_loss(M, q₂, data, exact_log_q_data_r₂)
end

In [50]:
# We want plots with (1) the lower bound error, (2) the actually uncorrected manifold error and (3) the corrected manifold error
# basepoint q₁
plot(1:d, [β(κ₁)^2 .* naive_tangent_distances_r₁, true_naive_distances_r₁, true_curvature_corrected_distances_r₁, exact_distances_r₁], label = ["theoretical lower bound" "naive" "curvature corrected" "exact"], ylims=(0,1), xlims=(1,d),xaxis=("approximation rank"), yaxis=(L"$\varepsilon_{rel}$"))
savefig("results/artificial1D_errors_by_rank_1.svg")
plot(1:d, [β(κ₁)^2 .* naive_tangent_distances_r₁ .+ 1e-4, true_naive_distances_r₁ .+ 1e-4, true_curvature_corrected_distances_r₁ .+ 1e-4, exact_distances_r₁ .+ 1e-4], label = ["theoretical lower bound" "naive" "curvature corrected" "exact"], ylims=(1e-4,1), xlims=(1,d), xaxis=("approximation rank"), yaxis=(L"$\varepsilon_{rel}$", :log), legend=:bottomleft)
savefig("results/artificial1D_logerrors_by_rank_1.svg")
# basepoint q₂
plot(1:d, [β(κ₂)^2 .* naive_tangent_distances_r₂, true_naive_distances_r₂, true_curvature_corrected_distances_r₂, exact_distances_r₂], label = ["theoretical lower bound" "naive" "curvature corrected" "exact"], ylims=(0,1), xlims=(1,d),xaxis=("approximation rank"), yaxis=(L"$\varepsilon_{rel}$"))
savefig("results/artificial1D_errors_by_rank_2.svg")
plot(1:d, [β(κ₂)^2 .* naive_tangent_distances_r₂ .+ 1e-4, true_naive_distances_r₂ .+ 1e-4, true_curvature_corrected_distances_r₂ .+ 1e-4, exact_distances_r₂ .+ 1e-4], label = ["theoretical lower bound" "naive" "curvature corrected" "exact"], ylims=(1e-4,1), xlims=(1,d), xaxis=("approximation rank"), yaxis=(L"$\varepsilon_{rel}$", :log), legend=:bottomleft)
savefig("results/artificial1D_logerrors_by_rank_2.svg")
for i in 1:d-1
    if i == 1
        plot(1:length(eCosts₁[1]), eCosts₁[1], label = "rank 1", ylims=(1e-4,1), yaxis=(L"$\varepsilon_{rel}$", :log))
    else
        plot!(1:length(eCosts₁[i]), eCosts₁[i], label = "rank $(i)", ylims=(1e-4,1), yaxis=(L"$\varepsilon_{rel}$", :log))
    end
end
savefig("results/artificial1D_exact_iterate_loss_1.svg")
for i in 1:d-1
    if i == 1
        plot(1:length(eCosts₂[1]), eCosts₂[1], label = "rank 1", ylims=(1e-4,1), yaxis=(L"$\varepsilon_{rel}$", :log))
    else
        plot!(1:length(eCosts₂[i]), eCosts₂[i], label = "rank $(i)", ylims=(1e-4,1), yaxis=(L"$\varepsilon_{rel}$", :log))
    end
end
savefig("results/artificial1D_exact_iterate_loss_2.svg")

"/Users/wdiepeveen/Documents/PhD/Projects/8 - Manifold-valued tensor decomposition/src/manifold-valued-tensors/experiments/1D/S2/results/artificial1D_exact_iterate_loss_2.svg"

In [49]:
# It would be nice to also have a plot that tells us something about the error in predicting the manifold loss (using CCL) and the actual loss 
# (1) for the naive approach (2) for the curvature corrected approach
# basepoint q₁
plot(1:d-1, (predicted_curvature_corrected_distances_r₁[1:end-1] .- true_curvature_corrected_distances_r₁[1:end-1] .+ 1e-16) ./ (curvature_corrected_tangent_distances_r₁[1:end-1] .* sqrt.(curvature_corrected_tangent_distances_r₁[1:end-1] .* ref_distance₁) .+ 1e-16), label=("curvature corrected"), xlims=(1,d-1),xaxis=("approximation rank"), yaxis=(L"$\delta_{rel}$"), color=3)
# plot(1:d, [(predicted_naive_distances_r₁ .+ 1e-16) ./ (true_naive_distances_r₁ .+ 1e-16), (predicted_curvature_corrected_distances_r₁ .+ 1e-16) ./ (true_curvature_corrected_distances_r₁ .+ 1e-16)], label = ["discrepancy in initialisation" "discrepancy in solutions"], xlims=(1,d),xaxis=("approximation rank"), yaxis=(L"$\delta_{rel}$"))
savefig("results/artificial1D_discrepancy_by_rank_1.svg")
# plot(1:d, [(predicted_naive_distances_r₁ .+ 1e-16) ./ (true_naive_distances_r₁ .+ 1e-16), (predicted_curvature_corrected_distances_r₁ .+ 1e-16) ./ (true_curvature_corrected_distances_r₁ .+ 1e-16)], label = ["discrepancy in initialisation" "discrepancy in solutions"], xlims=(1,d), xaxis=("approximation rank"), yaxis=(L"$\delta_{rel}$", :log), legend=:bottomleft)
# savefig("results/artificial1D_logdiscrepancy_by_rank_1.svg")
# basepoint q₂
plot(1:d-1, (predicted_curvature_corrected_distances_r₂[1:end-1] .- true_curvature_corrected_distances_r₂[1:end-1] .+ 1e-16) ./ (curvature_corrected_tangent_distances_r₂[1:end-1] .* sqrt.(curvature_corrected_tangent_distances_r₂[1:end-1] .* ref_distance₂) .+ 1e-16), label=("curvature corrected"), xlims=(1,d-1),xaxis=("approximation rank"), yaxis=(L"$\delta_{rel}$"), color=3)
# plot(1:d, [(predicted_naive_distances_r₂ .+ 1e-16) ./ (true_naive_distances_r₂ .+ 1e-16), (predicted_curvature_corrected_distances_r₂ .+ 1e-16) ./ (true_curvature_corrected_distances_r₂ .+ 1e-16)], label = ["discrepancy in initialisation" "discrepancy in solutions"], xlims=(1,d),xaxis=("approximation rank"), yaxis=(L"$\delta_{rel}$"))
savefig("results/artificial1D_discrepancy_by_rank_2.svg")
# plot(1:d, [(predicted_naive_distances_r₂ .+ 1e-16) ./ (true_naive_distances_r₂ .+ 1e-16), (predicted_curvature_corrected_distances_r₂ .+ 1e-16) ./ (true_curvature_corrected_distances_r₂ .+ 1e-16)], label = ["discrepancy in initialisation" "discrepancy in solutions"], xlims=(1,d), xaxis=("approximation rank"), yaxis=(L"$\delta_{rel}$", :log), legend=:bottomleft)
# savefig("results/artificial1D_logdiscrepancy_by_rank_2.svg")

"/Users/wdiepeveen/Documents/PhD/Projects/8 - Manifold-valued tensor decomposition/src/manifold-valued-tensors/experiments/1D/S2/results/artificial1D_discrepancy_by_rank_2.svg"

### Benchmark different methods ###

First benchmark for point q1

In [30]:
@benchmark naive_low_rank_approximation(M, q₁, data, 2)

BenchmarkTools.Trial: 5720 samples with 1 evaluation.
 Range (min … max):  533.462 μs …  51.545 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     740.916 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   862.213 μs ± 927.062 μs  ┊ GC (mean ± σ):  4.23% ± 6.15%

     █▅▁                                                         
  ▂▂▄███▅▅▅▅▅▆▆▅▅▄▄▄▄▃▃▃▃▃▂▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▂▂▁▁▂▂▂▁▂ ▃
  533 μs           Histogram: frequency by time         2.09 ms <

 Memory estimate: 333.58 KiB, allocs estimate: 522.

In [32]:
@benchmark curvature_corrected_low_rank_approximation(M, q₁, data, 2) 

BenchmarkTools.Trial: 2014 samples with 1 evaluation.
 Range (min … max):  1.642 ms … 52.966 ms  ┊ GC (min … max): 0.00% …  0.00%
 Time  (median):     2.153 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   2.467 ms ±  1.623 ms  ┊ GC (mean ± σ):  6.03% ± 10.90%

   █▆▂                                                        
  ▃█████▇▆▆▅▄▃▃▃▃▂▂▂▂▂▁▁▂▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▂▂▂▂▂▂▂▂▂ ▃
  1.64 ms        Histogram: frequency by time        9.02 ms <

 Memory estimate: 1.48 MiB, allocs estimate: 4616.

In [34]:
@benchmark exact_low_rank_approximation(M, q₁, data, 2; stepsize=1/100000, max_iter=1) 

BenchmarkTools.Trial: 961 samples with 1 evaluation.
 Range (min … max):  3.673 ms … 21.829 ms  ┊ GC (min … max):  0.00% … 76.55%
 Time  (median):     4.512 ms              ┊ GC (median):     0.00%
 Time  (mean ± σ):   5.187 ms ±  2.215 ms  ┊ GC (mean ± σ):  10.24% ± 15.60%

  ▃██▆▆▅▆▅▅▄▃▃▂▁                                              
  ████████████████▇▆▆▄▇▁▁▄▄▄▁▁▁▁▁▁▁▄▁▁▆▄▅▄▅▅▆▆▆▆▄▄▆▅▄▆▅▁▆▅▆▅ █
  3.67 ms      Histogram: log(frequency) by time     14.1 ms <

 Memory estimate: 4.54 MiB, allocs estimate: 13710.

In [36]:
@benchmark exact_low_rank_approximation(M, q₁, data, 2; stepsize=1/100000, max_iter=50)  

BenchmarkTools.Trial: 44 samples with 1 evaluation.
 Range (min … max):  102.806 ms … 125.776 ms  ┊ GC (min … max):  8.85% … 18.87%
 Time  (median):     113.591 ms               ┊ GC (median):    16.14%
 Time  (mean ± σ):   113.760 ms ±   5.521 ms  ┊ GC (mean ± σ):  14.83% ±  3.49%

                              █         ▂█                       
  █▅▁▁▅▁▁▅▅▁▁▅▁▁▅▁▁█▁▅█▁▁▁█▅▅▅█▅▁▁█▁▁▅▅▁██▅▁▅▅▁▅▁▅▁▁▅▁▁▁▁▅▁▁▁▁▅ ▁
  103 ms           Histogram: frequency by time          126 ms <

 Memory estimate: 120.39 MiB, allocs estimate: 348092.

In [38]:
nT₁ = []
nΣ₁ = []

for i in 1:d
    nbm₁ = @benchmark naive_low_rank_approximation(M, q₁, data, $i)
    push!(nT₁, mean(nbm₁).time / 1e9)
    push!(nΣ₁, std(nbm₁).time / 1e9)
end

In [39]:
ccT₁ = []
ccΣ₁ = []

for i in 1:d
    ccbm₁ = @benchmark curvature_corrected_low_rank_approximation(M, q₁, data, $i)
    push!(ccT₁, mean(ccbm₁).time / 1e9)
    push!(ccΣ₁, std(ccbm₁).time / 1e9)
end

In [40]:
eT1₁ = []
eΣ1₁ = []

for i in 1:d
    ebm1₁ = @benchmark exact_low_rank_approximation(M, q₁, data, $i; stepsize=1/100000, max_iter=1)
    push!(eT1₁, mean(ebm1₁).time / 1e9)
    push!(eΣ1₁, std(ebm1₁).time / 1e9)
end

In [41]:
eT₁ = []
eΣ₁ = []

for i in 1:d
    ebm₁ = @benchmark exact_low_rank_approximation(M, q₁, data, $i; stepsize=1/100000, max_iter=50)
    push!(eT₁, mean(ebm₁).time / 1e9)
    push!(eΣ₁, std(ebm₁).time / 1e9)
end

In [46]:
# methods above each other and results per rank in colums
println("tHOSVD" * prod([" & " * raw"$" * "$(Float16(nT₁[i]))" * raw"$" for i in 1:d]) * raw"\\ ")
println("CC-tHOSVD" * prod([" & " * raw"$" * "$(Float16(ccT₁[i]))" * raw"$" for i in 1:d]) * raw"\\ ")
println("MC-tHOSVD (1 iteration)" * prod([" & " * raw"$" * "$(Float16(eT1₁[i]))" * raw"$" for i in 1:d]) * raw"\\ ")
println("MC-tHOSVD (50 iteration)" * prod([" & " * raw"$" * "$(Float16(eT₁[i]))" * raw"$" for i in 1:d]) * raw"\\ ")

tHOSVD & $0.0007296$ & $0.000831$ & $0.0009346$ & $0.000925$ & $0.001057$ & $0.001043$\\ 
CC-tHOSVD & $0.002184$ & $0.002344$ & $0.002861$ & $0.002981$ & $0.0033$ & $0.00331$\\ 
MC-tHOSVD (1 iteration) & $0.004684$ & $0.005947$ & $0.005978$ & $0.00628$ & $0.006687$ & $0.004112$\\ 
MC-tHOSVD (50 iteration) & $0.11127$ & $0.1393$ & $0.13$ & $0.1368$ & $0.148$ & $0.004272$\\ 


Test point q2

In [31]:
@benchmark naive_low_rank_approximation(M, q₂, data, 2)

BenchmarkTools.Trial: 6767 samples with 1 evaluation.
 Range (min … max):  539.921 μs …   9.425 ms  ┊ GC (min … max): 0.00% … 91.97%
 Time  (median):     639.071 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   732.045 μs ± 545.274 μs  ┊ GC (mean ± σ):  4.93% ±  6.24%

       ▂██▇▆▅▄▄▃▃▂▂▂▁▁▁▁▁▁▂▁▁▁▁▁  ▁ ▁▁                          ▂
  ▇██▇▅███████████████████████████████▇████▇█▇█▇▇▇▇▆▆▆▆▆▆▅▆▅▅▃▆ █
  540 μs        Histogram: log(frequency) by time       1.26 ms <

 Memory estimate: 333.58 KiB, allocs estimate: 522.

In [33]:
@benchmark curvature_corrected_low_rank_approximation(M, q₂, data, 2) 

BenchmarkTools.Trial: 2140 samples with 1 evaluation.
 Range (min … max):  1.644 ms … 50.125 ms  ┊ GC (min … max): 0.00% …  0.00%
 Time  (median):     1.958 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   2.324 ms ±  1.567 ms  ┊ GC (mean ± σ):  6.39% ± 10.86%

  ▂█▆▅▄▄▄▃▂▂▁▁                                                
  █████████████▇▇▆▄▃▅▅▃▁▄▁▁▁▁▁▃▁▁▁▁▁▁▁▁▃▁▁▁▁▁▁▁▁▃▃▅▅▅▅▃▅▄▁▄▄ █
  1.64 ms      Histogram: log(frequency) by time     9.43 ms <

 Memory estimate: 1.48 MiB, allocs estimate: 4616.

In [35]:
@benchmark exact_low_rank_approximation(M, q₂, data, 2; stepsize=1/100000, max_iter=1) 

BenchmarkTools.Trial: 915 samples with 1 evaluation.
 Range (min … max):  3.739 ms … 62.237 ms  ┊ GC (min … max): 0.00% …  0.00%
 Time  (median):     4.827 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   5.449 ms ±  2.884 ms  ┊ GC (mean ± σ):  9.36% ± 15.16%

  ▄█▇▆▆▇▆▆▅▅▄▃▂                                              ▁
  ██████████████▇▇▇▅▄▇▄▁▁▁▁▄▁▁▄▄▄▄▅▆▅▅▄▇▄▆▇▅▇▆▅▅▅▆▄▅▁▁▆▁▁▄▁▅ █
  3.74 ms      Histogram: log(frequency) by time     15.3 ms <

 Memory estimate: 4.54 MiB, allocs estimate: 13710.

In [37]:
@benchmark exact_low_rank_approximation(M, q₂, data, 2; stepsize=1/100000, max_iter=50)  

BenchmarkTools.Trial: 44 samples with 1 evaluation.
 Range (min … max):  103.549 ms … 143.898 ms  ┊ GC (min … max):  8.93% … 17.38%
 Time  (median):     113.964 ms               ┊ GC (median):    15.89%
 Time  (mean ± σ):   115.199 ms ±   9.503 ms  ┊ GC (mean ± σ):  14.55% ±  3.70%

  ▄ ▄        █ ▁▄▁▁█▄▄▁                                          
  █▁█▁▆▆▆▆▆▁▁█▆████████▆▁▁▁▁▁▁▁▁▁▁▁▆▁▁▁▁▁▁▁▁▁▁▁▁▆▆▁▆▁▁▁▁▁▆▁▁▁▁▆ ▁
  104 ms           Histogram: frequency by time          144 ms <

 Memory estimate: 120.40 MiB, allocs estimate: 348092.

In [42]:
nT₂ = []
nΣ₂ = []

for i in 1:d
    nbm₂ = @benchmark naive_low_rank_approximation(M, q₂, data, $i)
    push!(nT₂, mean(nbm₂).time / 1e9)
    push!(nΣ₂, std(nbm₂).time / 1e9)
end

In [43]:
ccT₂ = []
ccΣ₂ = []

for i in 1:d
    ccbm₂ = @benchmark curvature_corrected_low_rank_approximation(M, q₂, data, $i)
    push!(ccT₂, mean(ccbm₂).time / 1e9)
    push!(ccΣ₂, std(ccbm₂).time / 1e9)
end

In [44]:
eT1₂ = []
eΣ1₂ = []

for i in 1:d
    ebm1₂ = @benchmark exact_low_rank_approximation(M, q₂, data, $i; stepsize=1/100000, max_iter=1)
    push!(eT1₂, mean(ebm1₂).time / 1e9)
    push!(eΣ1₂, std(ebm1₂).time / 1e9)
end

In [45]:
eT₂ = []
eΣ₂ = []

for i in 1:d
    ebm₂ = @benchmark exact_low_rank_approximation(M, q₂, data, $i; stepsize=1/100000, max_iter=50)
    push!(eT₂, mean(ebm₂).time / 1e9)
    push!(eΣ₂, std(ebm₂).time / 1e9)
end

In [47]:
# methods above each other and results per rank in colums
println("tHOSVD" * prod([" & " * raw"$" * "$(Float16(nT₂[i]))" * raw"$" for i in 1:d]) * raw"\\ ")
println("CC-tHOSVD" * prod([" & " * raw"$" * "$(Float16(ccT₂[i]))" * raw"$" for i in 1:d]) * raw"\\ ")
println("MC-tHOSVD (1 iteration)" * prod([" & " * raw"$" * "$(Float16(eT1₂[i]))" * raw"$" for i in 1:d]) * raw"\\ ")
println("MC-tHOSVD (50 iteration)" * prod([" & " * raw"$" * "$(Float16(eT₂[i]))" * raw"$" for i in 1:d]) * raw"\\ ")

tHOSVD & $0.0007234$ & $0.000776$ & $0.000988$ & $0.000912$ & $0.000971$ & $0.001128$\\ 
CC-tHOSVD & $0.001934$ & $0.002214$ & $0.002464$ & $0.002684$ & $0.002901$ & $0.00312$\\ 
MC-tHOSVD (1 iteration) & $0.00469$ & $0.00495$ & $0.005516$ & $0.006294$ & $0.00602$ & $0.003675$\\ 
MC-tHOSVD (50 iteration) & $0.1043$ & $0.1115$ & $0.1209$ & $0.128$ & $0.1375$ & $0.00368$\\ 
